# Algorithm 2: One-Versus-One (OVO) Soft-margin Gaussian Support Vector Machine (SVM)

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
import cvxopt
from cvxopt import matrix, solvers

In [2]:
# Imports data
file = open('zipcombo.dat','r')
tempdata = file.read().split('\n')
del tempdata[-1]

In [3]:
# Creates X and y from the data
X = np.zeros((len(tempdata), 256))
y = np.zeros((len(tempdata)))
for i in range(0,len(tempdata)):
    individ = tempdata[i].split(" ")
    del individ[-1]
    X[i] = np.array(individ[1:]).astype(np.float)
    y[i] = np.array(individ[0]).astype(np.float)
y = y.reshape(-1, 1)

In [5]:
# Splits data into 80% train and 20% test
def train_test_split(X, y):
    randsamp = np.random.rand(X.shape[0])
    split = randsamp < np.percentile(randsamp, 80)
    X_train = X[split].astype(np.float)
    y_train = y[split].astype(np.float) 
    X_test =  X[~split].astype(np.float) 
    y_test = y[~split].astype(np.float)
    return X_train, y_train, X_test, y_test

In [26]:
# Splits data into 5-folds for cross-validation
def five_fold_split(X):
    s = int(len(X)/5)
    ind = np.arange(len(X)).astype(int)
    train_index = np.zeros((5, len(X)-s))
    test_index = np.zeros((5, s))
    for i in range(0, 5):
        test_index[i] = ind[0+i*s:0+(i+1)*s]
        train_index[i] = np.delete(ind, test_index[i].astype(int))        
    return train_index, test_index

In [6]:
# Generates table of values
def df_generator(amean, astd, bmean, bstd, cmean, cstd, indexlabels, columnlabels, shape):
    str1 = np.char.add(np.char.mod('%.5f', amean), "+-")
    str1 = np.char.add(str1, np.char.mod('%.5f', astd))
    str2 = np.char.add(np.char.mod('%.5f', bmean), "+-")
    str2 = np.char.add(str2, np.char.mod('%.5f', bstd))
    str3 = np.char.add(np.char.mod('%.5f', cmean), "+-")
    str3 = np.char.add(str3, np.char.mod('%.5f', cstd))
    df = pd.DataFrame(data = np.array([str1, str2, str3]).reshape(shape), index = indexlabels, columns = columnlabels)
    return df

In [7]:
# Generates table of values
def df_generator2(amean, astd, bmean, bstd, indexlabels, columnlabels, shape):
    str1 = np.char.add(np.char.mod('%.5f', amean), "+-")
    str1 = np.char.add(str1, np.char.mod('%.5f', astd))
    str2 = np.char.add(np.char.mod('%.5f', bmean), "+-")
    str2 = np.char.add(str2, np.char.mod('%.5f', bstd))
    df = pd.DataFrame(data = np.array([str1, str2]).T, index = indexlabels, columns = columnlabels)
    return df

In [8]:
# Computes Gaussian Kernel
def kernel(Xi, Xt, c):
    Xiprime = np.sum(Xi**2, axis = 1, keepdims = True)
    Xtprime = np.sum(Xt.T**2, axis = 0, keepdims = True)
    texp = Xiprime+Xtprime-2*np.dot(Xi, Xt.T)
    K = np.exp(-c*texp)
    return K

In [9]:
# Uses quadratic programming (QP) to find SVM solution for one classifier
def train_one(X_train, y_train, K, C, c):
    N = X_train.shape[0]
    P = matrix(np.dot(y_train, y_train.T)*K)
    q = matrix(np.ones((N, 1))*-1)
    A = matrix(y_train.reshape(1, -1).astype('float'))
    bias = matrix(np.zeros(1))
    G = matrix(np.vstack((np.eye(N)*-1, np.eye(N))))
    h = matrix(np.hstack((np.zeros(N), np.ones(N)*C)))
    sol = solvers.qp(P, q, G, h, A, bias)
    lambdas = np.array(sol['x'])
    ind = (lambdas > 0.0001).flatten()
    sv_x = X_train[ind]
    sv_y = y_train[ind]
    sv_lambdas = lambdas[ind]
    sv_kernel = kernel(sv_x, sv_x, c)
    b = np.median(sv_y-np.sum(sv_kernel*sv_lambdas*sv_y,axis=0).reshape(-1, 1))    
    return sv_x, sv_y, sv_lambdas, b

In [10]:
# Trains all (K choose 2) classifiers
def train_all(X_train, y_train, c, C):
    k = len(np.unique(y_train))
    sv_x_arr = []
    sv_y_arr = []
    sv_lambdas_arr = []
    b_arr = []
    ind = 0
    K = kernel(X_train, X_train, c)
    for i in range(0, k):
        for j in range(i+1, k):
            ind = np.logical_or(y_train == i, y_train == j)
            ind = ind.flatten()
            ty = y_train[ind]
            ty[ty == j] = -1
            ty[ty == i] = 1
            tx = X_train[ind]
            tK = K[ind]
            tK = tK[:, ind]
            sv_x, sv_y, sv_lambdas, b = train_one(tx, ty, tK, C, c)
            sv_x_arr.append(sv_x)
            sv_y_arr.append(sv_y)
            sv_lambdas_arr.append(sv_lambdas)
            b_arr.append(b)
    return sv_x_arr, sv_y_arr, sv_lambdas_arr, b_arr

In [11]:
# Tests one classifier
def test_one(Xtest, sv_x, sv_y, sv_lambdas, b, c):
    K = kernel(sv_x, Xtest, c)
    y_hat = np.sum(np.multiply(K, sv_lambdas*sv_y.reshape(-1, 1)), axis = 0).reshape(-1, 1)+b
    y_hatsign = np.sign(y_hat)
    y_hatsign[np.where(y_hatsign == 0)] = -1
    return y_hatsign

In [12]:
# Tests using all (K choose 2) classifiers, makes final predictions, and computes error
def test_all(X_train, y_train, X_test, y_test, d, sv_x_arr, sv_y_arr, sv_lambdas_arr, b_arr):
    k = len(np.unique(y_test))
    confidence = np.zeros((int(k*(k-1)/2), len(X_test)))
    ind = 0
    for i in range(0, k):
        for j in range(i+1, k):
            y_hatsign = test_one(X_test, sv_x_arr[ind], sv_y_arr[ind], sv_lambdas_arr[ind], b_arr[ind], d)
            confidence[ind] = y_hatsign.reshape(y_hatsign.shape[0])
            ind += 1
    kconfidence = np.zeros((k, len(X_test)))
    for i in range(0, k):
        ind = 0
        for j in range(0, k):
            for l in range(j+1, k):
                if i == j:
                    kconfidence[i] += confidence[ind]
                elif i == l:
                    kconfidence[i] -= confidence[ind]
                ind += 1
    predictions = np.zeros(len(X_test))
    for i in range(0, len(X_test)):
        predictions[i] = np.argmax(kconfidence[:, i])
    mistakes = len(np.where(predictions != y_test.reshape(y_test.shape[0]))[0])
    return mistakes/len(X_test)

### Basic Results

In [14]:
# Runs training and testing for specified number of runs
def train_test(X, y, runs, d, C):
    Etrain = np.zeros((runs))
    Etest = np.zeros((runs))
    for e in range(0, runs):
        X_train, y_train, X_test, y_test = train_test_split(X, y)
        sv_x_arr, sv_y_arr, sv_lambdas_arr, b_arr = train_all(X_train, y_train, d, C)
        Etrain[e] = test_all(X_train, y_train, d, sv_x_arr, sv_y_arr, sv_lambdas_arr, b_arr)
        Etest[e] = test_all(X_test, y_test, d, sv_x_arr, sv_y_arr, sv_lambdas_arr, b_arr)
    return np.mean(Etrain), np.std(Etrain), np.mean(Etest), np.std(Etest)

In [47]:
# Parameter initialization
median = np.median(distance.cdist(X, X))
c = 1/median
cS = np.arange(c-0.055, c-0.04, 0.005)
print(cS)
CS=[1, 10, 100]

[0.00901815 0.01401815 0.01901815]


In [ ]:
Etrain = np.zeros((6))
stdtrain = np.zeros((6))
Etest = np.zeros((6))
stdtest = np.zeros((6))
ind = 0
for i in range(len(CS)):
    for j in range(len(cS)):
        Etrain[ind], stdtrain[ind], Etest[ind], stdtest[ind] = train_test(X, y, 20, cS[j], CS[i])
        ind += 1

In [25]:
df = df_generator2(Etrain, stdtrain, Etest, stdtest, ["C=1,c=0.009", "C=1,c=0.014", 
                                                      "C=1,c=0.019","C=10,c=0.009", "C=10,c=0.014", "C=10,c=0.019", "C=100,c=0.009", "C=100,c=0.014", "C=100,c=0.019"], 
                   ["Train Error","Test Error"], (9, 2))
df.style

,Train Error,Test Error
"C=1,c=0.009",0.00477+-0.00038,0.02403+-0.00257
"C=1,c=0.014",0.00212+-0.00028,0.02304+-0.00382
"C=1,c=0.019",0.00138+-0.00020,0.02710+-0.00304
"C=10,c=0.009",0.00017+-0.00010,0.02242+-0.00318
"C=10,c=0.014",0.00022+-0.00009,0.02355+-0.00299
"C=10,c=0.019",0.00025+-0.00005,0.02460+-0.00293
"C=100,c=0.009",0.00011+-0.00005,0.02169+-0.00292
"C=100,c=0.014",0.00012+-0.00004,0.02250+-0.00223
"C=100,c=0.019",0.00009+-0.00006,0.02430+-0.00304


### Cross-Validation

In [15]:
# Performs 5-fold cross validation
def cross_validation(X_train, y_train, CS, cS):
    train_in, test_in = five_fold_split(X_train)
    minE = 1000
    bestc = 1
    bestC = 1
    for C in CS:
        for c in cS:
            for i in range(0, 5):
                train_index = train_in[i].astype(int)
                test_index = test_in[i].astype(int)
                X_trainfold = X_train[train_index[0]:train_index[len(train_index)-1]]
                y_trainfold = y_train[train_index[0]:train_index[len(train_index)-1]]
                sv_x_arr, sv_y_arr, sv_lambdas_arr, b_arr = train_all(X_trainfold, y_trainfold, c, C)
                X_testfold = X_train[test_index[0]:test_index[len(test_index)-1]]
                y_testfold = y_train[test_index[0]:test_index[len(test_index)-1]]
                foldE = test_all(X_testfold, y_testfold, c, sv_x_arr, sv_y_arr, sv_lambdas_arr, b_arr)
            if foldE < minE:
                bestC = C
                bestc = c
                minE = foldE
    return bestc, bestC

In [16]:
# Performs cross validation on train data and uses optimal parameters C*, c* to train
# and test full dataset
def cross_train_test(X, y, runs, param):
    Etest = np.zeros((runs))
    cstar = np.zeros((runs))
    for e in range(0, runs):
        X_train, y_train, X_test, y_test = train_test_split(X, y)
        bestc, bestC = cross_validation(X_train, y_train, CS, cS)
        sv_x_arr, sv_y_arr, sv_lambdas_arr, b_arr = train_all(X_train, y_train, bestc, bestC)
        Etest[e] = test_all(Xtest, ytest, bestc, sv_x_arr, sv_y_arr, sv_lambdas_arr, b_arr)
        cstar[e] = bestc  
        Cstar[e] = bestC      
    return Etest, cstar, Cstar

In [ ]:
E, bestc, bestC = cross_train_test(X, y, 20, list(range(1, 8)))

In [30]:
df2 = df_generator(np.mean(bestC), np.std(bestC), np.mean(bestc), np.std(bestc), np.mean(E), np.std(E), 
                   [""],["Mean C", "Mean c", "Mean Test Error"], (1, 3))
df2.style                

,Mean C,Mean c,Mean Test Error
,95.50000+-19.61505,0.00902+-0.00000,0.02183+-0.00346
